In [ ]:
import numpy as np
import json
import time
import datetime
from osgeo import gdal
import matplotlib.pyplot as plt
from area import area
import pickle
import geojson

from shapely import geometry
from skimage.morphology import watershed
from skimage.feature import peak_local_max
from skimage import measure
from pyproj import Proj
from scipy import ndimage, spatial
from scipy.signal import fftconvolve
from PIL import Image, ImageDraw
import requests

**Geolocate Cities and unk things**

In [ ]:
cities = json.load(open('all_cities.geojson','r'))

In [ ]:
with open('LK_google_maps.txt','rb') as f:
    API_KEY = str(f.read())[2:-1] #hacky shit
    print (API_KEY)

In [ ]:
def get_addy(latlon,API_KEY):
    #address_q = ','.join(address.split('\n')).replace(' ','+')
    #print(address_q)
    #https://maps.googleapis.com/maps/api/geocode/json?latlng=40.714224,-73.961452&key=YOUR_API_KEY
    urlstr = """https://maps.googleapis.com/maps/api/geocode/json?latlng="""+str(latlon[0])+""","""+str(latlon[1])+"""&key=""" + API_KEY
    
    #print (urlstr)
    
    r = requests.get(urlstr)
    data = r.json()
    return data
    
    #with urllib.request.urlopen(urlstr) as response:
    #    print(dir(response))
    #    print (response.readable())
    #    print (response.read())
    #    soup = response.read1()
    #    data = json.loads(soup.decode('utf-8'))


    #lat = float((data['results'][0]['geometry']['location']['lat']))
    #lon = float((data['results'][0]['geometry']['location']['lng']))

    #return (lat,lon)

In [ ]:
pt = [49.0,1.0]
res = get_addy(pt,API_KEY)
print (len(res['results']))
print (res['results'][0])

In [ ]:
for c in cities['features']:
    #print (c['properties'])
    c_poly = geometry.Polygon(c['geometry']['coordinates'][0])
    lon,lat = c_poly.centroid.xy
    pt = [lat[0],lon[0]]

    res = get_addy(pt, API_KEY)
    #print (len(res['results']))
    
    flag = False
    city_geocode = None
    country_geocode = None
    
    for priority_component in ['colloquial_area', 'postal_town','locality','administrative_area_level_2']:
        if flag==False:
            for r in res['results']:
                if priority_component in r['types']:
                    for component in r['address_components']:
                        if priority_component in component['types']:
                            city_geocode = component['long_name']
                        if 'country' in component['types']:
                            country_geocode = component['short_name']
                    flag==True
                    
    c['properties'] = json.loads(c['properties'])
    c['properties']['name'] =str(city_geocode)+', '+str(country_geocode)
    c['properties']['iso_2'] = str(country_geocode)
    c['properties']['type']='city_cluster'
    print (c['properties']['name'])

In [ ]:
json.dump(cities,open('all_cities_fts.geojson','w'))

**clean up Nones**

In [ ]:
for c in cities['features']:
    if c['properties']['iso_2']=='None':
        #print (c)
        c_poly = geometry.Polygon(c['geometry']['coordinates'][0])
        lon,lat = c_poly.centroid.xy
        pt = [lat[0],lon[0]]
        print (*pt)
        city_name = input('city name')
        country_iso2 = input('country iso2')
        c['properties']['name'] = str(city_name)+', '+str(country_iso2)
        c['properties']['iso_2'] = str(country_iso2)

In [ ]:
json.dump(cities,open('all_cities_fts.geojson','w'))

### Region Clusters

In [ ]:
clust_unique =  pickle.load(open('unique_clusters.pickle','rb'))
clusters = pickle.load(open('features_out.pickle','rb'))
names_dict = pickle.load(open('names_dict.pickle','rb'))

In [ ]:
for ii_c, c in enumerate(clusters):
    c['properties'] = json.loads(c['properties'])
    c['properties']['unique_index'] = clust_unique[ii_c+1]
    c['properties']['name'] =names_dict[clust_unique[ii_c+1]]
    c['properties']['type']='region_cluster'
    
    
    for k,v in c['properties'].items():
        #print (k[0:5],v)
        if ((k[0:6]=='co_pop') and (v<0)):
            c['properties'][k]=0
            
    c['properties']['popcount_clust'] = sum([v for k,v in c['properties'].items() if k[0:6]=='co_pop'])
        
    print(c['properties'])

In [ ]:
json.dump(geojson.FeatureCollection(clusters),open('clusters_fc.geojson','w'))

In [ ]:
all_fc = geojson.FeatureCollection(cities['features']+clusters)

In [ ]:
json.dump(all_fc,open('demand_nodes.geojson','w'))